<a href="https://colab.research.google.com/github/Shoh0428/3DCNN_with_Tensorflow/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import numpy as np
import os
from google.colab import drive

In [2]:
drive.mount('/content/drive')

content_folder = '/content/drive/My Drive/content'
advertisement_folder = '/content/drive/My Drive/Advertisement'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def process_video(video_path, max_frames):
    frames = []

    for frame_file in os.listdir(video_path):
        frame_path = os.path.join(video_path, frame_file)

        if frame_file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            frame = load_img(frame_path, target_size=(224, 224))
            frame = img_to_array(frame)
            frame = frame / 255.0
            frames.append(frame)

    while len(frames) < max_frames:
        frames.append(np.zeros((224, 224, 3)))

    return np.array(frames)



In [4]:
max_frames_content = max(len(os.listdir(os.path.join(content_folder, video))) for video in os.listdir(content_folder) if os.path.isdir(os.path.join(content_folder, video)))
max_frames_advertisement = max(len(os.listdir(os.path.join(advertisement_folder, video))) for video in os.listdir(advertisement_folder) if os.path.isdir(os.path.join(advertisement_folder, video)))
max_frames = max(max_frames_content, max_frames_advertisement)


In [5]:
content_videos = [process_video(os.path.join(content_folder, video), max_frames) for video in os.listdir(content_folder) if os.path.isdir(os.path.join(content_folder, video))]
content_labels = [0] * len(content_videos)

advertisement_videos = [process_video(os.path.join(advertisement_folder, video), max_frames) for video in os.listdir(advertisement_folder) if os.path.isdir(os.path.join(advertisement_folder, video))]
advertisement_labels = [1] * len(advertisement_videos)


In [6]:
X = np.concatenate([content_videos, advertisement_videos])
y = np.concatenate([content_labels, advertisement_labels])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
num_frames = max_frames

In [9]:
model = models.Sequential([
    layers.Conv3D(32, (3, 3, 3), activation='relu', input_shape=(num_frames, 224, 224, 3)),
    layers.MaxPooling3D((2, 2, 2)),
    layers.Conv3D(64, (3, 3, 3), activation='relu'),
layers.MaxPooling3D((2, 2, 2)),
layers.Flatten(),
layers.Dense(64, activation='relu'),
layers.Dense(1, activation='sigmoid')
])

In [10]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')




In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
